# Heroes of Pymoli Data Analysis

#3 Observed Trends

-  Heroes of Pymoli is largely dominated by males players (i.e., Males: 82%, Females: 16%, Other: 2%)
-  In-game purchases are broken out in nearly the same ratio as the gender percentages. 
-  In-game purchases are not a huge money maker. Only 4,879 dollars have been spent on in-game items and no one player has spent more than 20 dollars.
-  Of the in-game items, the Bone Crushing Silver Skewer is most profitable, having generated roughly 68 dollars in total purchase value.

In [2]:
import pandas as pd
import numpy as np
import json

In [145]:
# read in Players_Complete csv file
total_player_df = pd.read_csv('players_complete.csv')

# read purchase data json file
purchase_df = pd.read_json('purchase_data.json')
purchase2_df = pd.read_json('purchase_data2.json')
purchase3_df = pd.read_json('purchase_data_3.json~HEAD')

# read in items inventory
items_inventory = pd.read_csv('items_complete.csv')

In [146]:
## Player Count
# Change Column Header from SN to Screen Name
total_player_df = total_player_df.rename(columns={'SN':'Screen Name'})
total_player_df.head()

,Player ID,Screen Name,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male


In [147]:
## Player Count
# Count the total number of unique players
player_count = len(total_player_df['Screen Name'].unique())

total_players = pd.DataFrame({'Total Players':[player_count]})
total_players

,Total Players
0,1161


In [165]:
## Gender Demographics
# Identify percentage of male, female, and other/non-disclosed players

male_players = total_player_df['Gender'].value_counts()['Male']
perc_male = (male_players / player_count) * 100

female_players = total_player_df['Gender'].value_counts()['Female']
perc_female = (female_players / player_count) * 100

other_players = player_count - male_players - female_players
perc_other = (other_players / player_count) * 100

gender_demographics = pd.DataFrame({"Male Players": [male_players],
                                   "Percent Male": [perc_male],
                                   "Female Players": [female_players],
                                   "Percent Female": [perc_female],
                                   "Other Players": [other_players],
                                   "Percent Other":[perc_other],
})

gender_demographics = gender_demographics[['Male Players','Percent Male','Female Players','Percent Female','Other Players','Percent Other']]
#gender_demographics = gender_demographics.round(2)
gender_demographics.style.format({"Percent Male": "{:.2f}%", "Percent Female": "{:.2f}%", "Percent Other": "{:.2f}%"})


,Male Players,Percent Male,Female Players,Percent Female,Other Players,Percent Other
0,954,82.17%,187,16.11%,20,1.72%


In [163]:
## Purchasing Analysis (TOTAL)
# Concatenate the 3 json files
all_purchases = pd.concat([purchase_df, purchase2_df, purchase3_df], ignore_index=True)

# Number of Unique Items
#unique_items = all_purchase['Item Name'].count()
unique_items = len(all_purchases["Item Name"].unique())

# Average purchase price
avg_purchase_price = all_purchases['Price'].mean()

# Total purchases
total_purchases = len(all_purchases)

# Total revenue
total_revenue = all_purchases['Price'].sum()

# Create DataFrame
purchase_analysis_total = pd.DataFrame({"Number of Unique Items": [unique_items],
                                   "Average Price": [avg_purchase_price],
                                   "Number of Purchases": [total_purchases],
                                   "Total Revenue": [total_revenue]
})

purchase_analysis_total = purchase_analysis_total[['Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue']]
purchase_analysis_total.style.format({"Average Price": "${:.2f}", "Total Revenue": "${:.2f}"})



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,180,$2.98,1638,$4879.60


In [160]:
## Purchasing Analysis (GENDER) 
# Identify purchase count by M/F/O
male_purchase_count = all_purchases['Gender'].value_counts()['Male']
female_purchase_count = all_purchases['Gender'].value_counts()['Female']
other_purchase_count = total_purchases - male_purchase_count - female_purchase_count

# Average Purchase Price and Total Revenue by M/F/O
# males
male_purchases = all_purchases.loc[all_purchases['Gender'] == 'Male', :]
male_avg_price = male_purchases['Price'].mean()
male_revenue = male_purchases['Price'].sum()

# females
female_purchases = all_purchases.loc[all_purchases['Gender'] == 'Female', :]
female_avg_price = female_purchases['Price'].mean()
female_revenue = female_purchases['Price'].sum()

#others
other_purchases_minus_female = all_purchases.loc[all_purchases['Gender'] != 'Female', :]
other_purchases = other_purchases_minus_female.loc[other_purchases_minus_female['Gender'] != 'Male', :]
other_avg_price = other_purchases['Price'].mean()
other_revenue = total_revenue - male_revenue - female_revenue

# Normalized Totals of M/F/O

# Create DataFrame
purchase_analysis_gender = pd.DataFrame({"Gender":['Male','Female','Other'],
                                         "Purchase Count":[male_purchase_count,female_purchase_count,other_purchase_count],
                                         "Average Price":[male_avg_price,female_avg_price, other_avg_price],
                                         "Total Value":[male_revenue, female_revenue, other_revenue]})

purchase_analysis_gender = purchase_analysis_gender[['Gender','Purchase Count','Average Price','Total Value']]
purchase_analysis_gender.style.format({"Average Price": "${:.2f}", "Total Value": "${:.2f}"})
|

,Gender,Purchase Count,Average Price,Total Value
0,Male,1339,$2.99,$3997.79
1,Female,279,$2.94,$820.09
2,Other,20,$3.09,$61.72


In [166]:
## Age Demographics
# Create bins of 4 years
# By bin identify: Purchase Count, Avg Purch Price, Total Purch Value, Normalized Totals
bins = [0,4,8,12,16,20,24,28,32,36,40,44,48,52]
group_names = ['0 to 4','4 to 8','8 to 12','12 to 16','16 to 20','20 to 24','24 to 28','28 to 32','32 to 36','36 to 40','40 to 44','44 to 48','48 to 52']
pd.cut(all_purchases['Age'], bins, labels=group_names)
all_purchases['Age Bracket'] = pd.cut(all_purchases['Age'], bins, labels=group_names)

purchase_groups = all_purchases[['Age Bracket','Price','Item Name']]

group_total_purchase = purchase_groups.groupby('Age Bracket')['Price'].sum().to_frame()
group_purchase_count = purchase_groups.groupby('Age Bracket')['Item Name'].count().to_frame()
group_purchase_avg = purchase_groups.groupby('Age Bracket')['Price'].mean().to_frame()

group_total_purchase.columns=["Age Bracket"]
join_one = group_total_purchase.join(group_purchase_count, how="left")
join_one.columns=["Total Purchase Value", "Purchase Count"]

join_two = join_one.join(group_purchase_avg, how="inner")
join_two.columns=["Total Purchase Value", "Purchase Count", "Average Purchase Price"]

age_demo_df = join_two[["Total Purchase Value","Purchase Count", "Average Purchase Price"]]
age_demo_final = age_demo_df.sort_values('Total Purchase Value', ascending=False).head()
age_demo_final.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})


,Total Purchase Value,Purchase Count,Average Purchase Price
Age Bracket,,,
20 to 24,$1494.48,503,$2.97
16 to 20,$981.50,332,$2.96
24 to 28,$700.17,232,$3.02
12 to 16,$467.25,160,$2.92
28 to 32,$384.69,127,$3.03


In [167]:
## Top Spenders
# Identify the top 5 spenders in the game by total purchase value (list: SN, Purch Count, Avg Purch Price, Total Purch. Price)
top_spenders = all_purchases[['Price','SN']]

sn_total_purchase = top_spenders.groupby('SN')['Price'].sum().to_frame()
sn_purchase_count = top_spenders.groupby('SN')['Price'].count().to_frame()
sn_purchase_avg = top_spenders.groupby('SN')['Price'].mean().to_frame()

sn_total_purchase.columns=["Total Purchase Value"]
join_one = sn_total_purchase.join(sn_purchase_count, how="left")
join_one.columns=["Total Purchase Value", "Purchase Count"]

join_two = join_one.join(sn_purchase_avg, how="inner")
join_two.columns=["Total Purchase Value", "Purchase Count", "Average Purchase Price"]

top_spenders_df = join_two[["Total Purchase Value", "Purchase Count", "Average Purchase Price"]]
top_spenders_final = top_spenders_df.sort_values('Total Purchase Value', ascending=False).head()
top_spenders_final.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})


,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Aerithllora36,$19.14,6,$3.19
Sondim43,$17.81,5,$3.56
Rinallorap73,$17.50,5,$3.50
Undirrala66,$17.06,5,$3.41
Arithllorin55,$16.80,4,$4.20


In [153]:
## Most Popular Items
# Identify the top 5 items by purchase count (list: Name, Purch Count, Price, Total Purch Value)
popular_items = all_purchases[['Item ID','Item Name','Price']]

item_purchase_count = popular_items.groupby('Item ID')['Item Name'].count().to_frame()
item_total_purchase = popular_items.groupby('Item ID')['Price'].sum().to_frame()
item_price = popular_items.groupby('Item ID')['Price'].head().to_frame()
item_name = popular_items.groupby('Item ID')['Item Name'].head().to_frame()

item_purchase_count.columns=["Item Purchase Count"]
join_one = item_purchase_count.join(item_total_purchase, how="left")
join_one.columns=["Item Purchase Count", "Total Purchase Value"]

join_two = join_one.join(item_price, how="inner")
join_two.columns=["Item Purchase Count", "Total Purchase Value", "Item Price"]

join_three = join_two.join(item_name, how="inner")
join_three.columns=["Item Purchase Count", "Total Purchase Value", "Item Price","Item Name"]

popular_items_df = join_three[["Item Name","Item Purchase Count","Total Purchase Value", "Item Price"]]
popular_items_final = popular_items_df.sort_values('Item Purchase Count', ascending=False).head()
popular_items_final.style.format({"Total Purchase Value": "${:.2f}", "Item Price": "${:.2f}"})



,Item Name,Item Purchase Count,Total Purchase Value,Item Price
111,Azurewrath,16,$62.23,$2.22
31,"Shadow Strike, Glory of Ending Hope",16,$35.16,$1.93
92,Reaper's Toll,15,$30.34,$4.56
91,"Malice, Legacy of the Queen",15,$41.74,$2.38
84,"Thorn, Satchel of Dark Souls",15,$43.89,$4.51


In [159]:
## Most Profitable Items
# Identify the top 5 items by purchase value (list: Name, Purch Count, Price, Total Purch Value)
profitable_items = all_purchases[['Item ID','Item Name','Price']]

profit_item_purchase_count = profitable_items.groupby('Item ID')['Item Name'].count().to_frame()
profit_item_total_purchase = profitable_items.groupby('Item ID')['Price'].sum().to_frame()
profit_item_price = profitable_items.groupby('Item ID')['Price'].head().to_frame()
profit_item_name = profitable_items.groupby('Item ID')['Item Name'].head().to_frame()

profit_item_total_purchase.columns=['Total Purchase Value']
join_one = profit_item_purchase_count.join(profit_item_total_purchase, how="left")
join_one.columns=["Item Purchase Count", "Total Purchase Value"]

join_two = join_one.join(profit_item_price, how="inner")
join_two.columns=["Item Purchase Count", "Total Purchase Value", "Item Price"]

join_three = join_two.join(profit_item_name, how="inner")
join_three.columns=["Item Purchase Count", "Total Purchase Value", "Item Price","Item Name"]

profitable_items_df = join_three[["Item Name","Total Purchase Value","Item Purchase Count", "Item Price"]]
profitable_items_final = profitable_items_df.sort_values('Total Purchase Value', ascending=False).head()
profitable_items_final.style.format({"Total Purchase Value": "${:.2f}", "Item Price": "${:.2f}"})


,Item Name,Total Purchase Value,Item Purchase Count,Item Price
52,Bone Crushing Silver Skewer,$68.05,15,$3.37
93,Netherbane,$63.42,14,$1.48
111,Azurewrath,$62.23,16,$2.22
49,Severance,$58.03,13,$1.85
107,Spectral Diamond Doomblade,$52.87,13,$4.25
